# Camera Calibration
## Computer Vision and Image Processing - Lab Session 3
### Prof: Luigi Di Stefano, luigi.distefano@unibo.it
### Tutor: Pierluigi Zama Ramirez, pierluigi.zama@unibo.it

Camera calibration is the process whereby all parameters defining the camera
model are estimated for a specific camera device.

The pinhole camera model is represented by the Perspective Projection Matrix
(PPM), which in turn can be decomposed into 3 independent tokens: intrinsic
parameter matrix (A), rotation matrix (R) and translation vector (T). Depending on
the application, either the PPM only or also its independent components (A, R, T)
need to be estimated.

Many camera calibration algorithms do exist. The basic process, though, relies
always on setting up a linear system of equations given a set of known 3D-2D
correspondences, so as to then solve for the unknown camera parameters.

To obtain the required correspondences specific physical objects (referred to as
calibration targets) having easily detectable features (such as e.g. chessboard or
dot patterns) are typically deployed.

Camera calibration approaches can be split into two main categories:

* Those relying on a single image featuring several (at least  2) planes containing a known pattern. 
* Those relying on several (at least 3) different images of one given planar pattern. 

In practise, it is difficult to build accurate targets containing multiple planes, while an accurate planar target can be attained rather easily. 

Implementing a camera calibration software requires a significant effort. However, the main Computer Vision toolbox include specific functions (OpenCV, Matlab CC Toolbox, Halcon.) 

## Implementing **Camera Calibration** with *OpenCV* and *Python*

First of all we need to import the usual libraries:

In [66]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

Then we need to define both the size of the square (measure it direcly using a ruler on the printed chessboard).

In [ ]:
square_size = 26.5 #mm

Then define the number of inner corner per row and column. We will call it *pattern_size*:

In [6]:
pattern_size = (9,6) # number of inner corner

Let us create a dictionary of the path to all the calibration images. (Take new ones if you want to calibrate your own camera or use the pictures contained inside *"calibration/chessboards"*).

The **minimum** number of images to calibrate a camera is __3__. A rule of thumb is to take **at least 12** pictures because it is possible that you will not be able to detect the chessboard in all of them and it will make calibration results more robusts.

Moreover, try to take both pictures with **several rotations of the chessboard**.

In [17]:
dirname = "calibration/chessboards/"
img_names = [dirname + str(i) + ".jpg" for i in range(13)]
print(img_names)

['calibration/chessboards/0.jpg', 'calibration/chessboards/1.jpg', 'calibration/chessboards/2.jpg', 'calibration/chessboards/3.jpg', 'calibration/chessboards/4.jpg', 'calibration/chessboards/5.jpg', 'calibration/chessboards/6.jpg', 'calibration/chessboards/7.jpg', 'calibration/chessboards/8.jpg', 'calibration/chessboards/9.jpg', 'calibration/chessboards/10.jpg', 'calibration/chessboards/11.jpg', 'calibration/chessboards/12.jpg']


During calibration process we need 2D-3D correspondeces. Let us create the 3D coordinate for each corner of the chessboard. 

We know that 3D coordinates of a corner $c_{i,j}$ will be *i x square_size* and *j x square_size*. 

The total number of corner is *rows x columns* of the pattern size.

We need to produce the 3D coordinate for each corner. For this operation is really useful *np.indices(shape)* which returns an array containing a rows x column array containing the 2D indices of that array:

In [32]:
indices = np.indices(pattern_size, dtype=np.float32)
print("Shape of indices: " , indices.shape)
print(indices)

Shape of indices:  (2, 9, 6)
[[[0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1.]
  [2. 2. 2. 2. 2. 2.]
  [3. 3. 3. 3. 3. 3.]
  [4. 4. 4. 4. 4. 4.]
  [5. 5. 5. 5. 5. 5.]
  [6. 6. 6. 6. 6. 6.]
  [7. 7. 7. 7. 7. 7.]
  [8. 8. 8. 8. 8. 8.]]

 [[0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]
  [0. 1. 2. 3. 4. 5.]]]


Indices contains nothing less than the coordinate of the pattern!

**N.B** we need it in **float** type because later we will need to multiply it with float numbers.

Let us see for instance the x and y indices of the position 1,1 in the grid:

In [33]:
print(indices[:,1,1])

[1. 1.]


(1,1) of course!

Now we know that each the distance between the corners is exactly 26.5mm (square_size). So we have to multiply this indices by square_size to get the real 3D x,y coordinates:

In [34]:
indices *= square_size
print(indices)

[[[  0.    0.    0.    0.    0.    0. ]
  [ 26.5  26.5  26.5  26.5  26.5  26.5]
  [ 53.   53.   53.   53.   53.   53. ]
  [ 79.5  79.5  79.5  79.5  79.5  79.5]
  [106.  106.  106.  106.  106.  106. ]
  [132.5 132.5 132.5 132.5 132.5 132.5]
  [159.  159.  159.  159.  159.  159. ]
  [185.5 185.5 185.5 185.5 185.5 185.5]
  [212.  212.  212.  212.  212.  212. ]]

 [[  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]
  [  0.   26.5  53.   79.5 106.  132.5]]]


We got the x,y coordinates of each corner in the world reference system ! (assumed that the first is in position (0mm,0mm)). For instance let us try to plot again the 1,1 corner position:

In [45]:
print(indices[:,1,1])

[26.5 26.5]


It is in position 26.5,26.5 mm in the world! Moreover, during the camera calibration we assume that all the pixels belonging to the chessboard lie in the same plane with z=0! 

So the 3D coordinate in the WRS of each corner is just x,y,0!

Let us create an array for containing each 3D cordinate. It will have shape *rows x columns x 3*  where rows and columns are refered to the pattern size while 3 is because we need x,y,z values.

In [46]:
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
print(pattern_points.shape)

(54, 3)


Now let us assing each x,y to the corresponding corner. We need to transpose the (2,9,6) to become a (6,9,2) array and then reshape it to (54,3).

In [50]:
coords_3D = indices.T
print("Transpose shape: " , coords_3D.shape)
coords_3D = coords_3D.reshape(-1, 2)
print(coords_3D.shape)
pattern_points[:, :2] = coords_3D

Transpose shape:  (6, 9, 2)
(54, 2)


Now that we have the 3D coordinate of each corner in the world reference system we need to get the cooresponding 2D coordinate of each corner in the chessboard images. 

Let us first load a sample image to understand how to detect corners. It need to be loaded **GRAYSCALE**:

In [62]:
img = cv2.imread("calibration/chessboards/0.jpg",cv2.IMREAD_GRAYSCALE)

We can use **cv2.findChessboardCorners(img, pattern_size)** to detect corner in an image. The functions will return a boolean value **found** and the list of the **corners** 2D coordinate in the image. **found** will be true if and only if all the 9x6 corners will be detected in the image. If the image is too dark or too bright the algorithm may fail to detect corners and **found** would be false. 

**N.B** If you pass the wrong pattern_size the algorithm will look for corners that are not present in the image and the method will be really slow with a **found=False** result.

In [63]:
found, corners = cv2.findChessboardCorners(img, pattern_size)
print("Found: " , found)
print("2D image coordinate of corners: ", corners)

Found:  True
2D image coordinate of corners:  [[[2774.       697.     ]]

 [[2752.3247   902.5533 ]]

 [[2732.7363  1102.0706 ]]

 [[2709.1055  1295.9447 ]]

 [[2689.049   1479.8164 ]]

 [[2673.347   1661.2803 ]]

 [[2654.8494  1841.0714 ]]

 [[2638.777   2010.5115 ]]

 [[2621.      2180.5    ]]

 [[2591.4824   684.34155]]

 [[2570.       895.     ]]

 [[2548.9783  1100.2012 ]]

 [[2532.4688  1296.8704 ]]

 [[2514.9172  1482.3511 ]]

 [[2497.5     1667.     ]]

 [[2482.2644  1847.2081 ]]

 [[2468.916   2023.2083 ]]

 [[2454.9106  2191.0999 ]]

 [[2398.5      675.5    ]]

 [[2380.5      890.     ]]

 [[2362.0938  1094.2607 ]]

 [[2347.9604  1294.852  ]]

 [[2332.5     1487.     ]]

 [[2317.7656  1674.1627 ]]

 [[2304.832   1852.8756 ]]

 [[2292.0693  2033.8956 ]]

 [[2280.4412  2204.0005 ]]

 [[2197.4026   667.12256]]

 [[2184.5      883.5    ]]

 [[2171.      1093.     ]]

 [[2158.5     1295.     ]]

 [[2144.8948  1491.245  ]]

 [[2134.0474  1676.0415 ]]

 [[2122.8203  1864.1527 ]]

 [

Even if we **found** th corners, they may be not accurate. To refine the results we can call **cv2.cornerSubPix**. See in OpenCV for more detail about this function. *documentation*.

In [64]:
if found:
    #Refining corner position to subpixel iteratively until criteria max_count=30 or criteria_eps_error=1 is sutisfied
    term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 1)
    #Image Corners 
    cv2.cornerSubPix(img, corners, (5, 5), (-1, -1), term)

Let us visualize the founded corners. Follow from red to blu lines to go from (0,0) to (9,6) corner:

In [72]:
vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
cv2.drawChessboardCorners(vis, pattern_size, corners, found)
plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.show()

When we calibrate a camera we need to repeat this operation on all calibration images so I will define a function to process images which return the pairs of (2D,3D) points:

In [76]:
def processImage(fn):
    print('processing {}'.format(fn))
    img = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        print("Failed to load", fn)
        return None

    found, corners = cv2.findChessboardCorners(img, pattern_size)

    if found:
        #Refining corner position to subpixel iteratively until criteria  max_count=30 or criteria_eps_error=1 is sutisfied
        term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 5, 1)
        #Image Corners 
        cv2.cornerSubPix(img, corners, (5, 5), (-1, -1), term)

    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.drawChessboardCorners(vis, pattern_size, corners, found)
    
    image_name = fn.split("/")[-1]
    basename, ext = image_name.split(".")
    outfile = os.path.join("calibration/output/", basename + '_chess.png')
    cv2.imwrite(outfile, vis)

    if not found:
        print('chessboard not found')
        return None

    print('           %s... OK' % fn)
    return (corners.reshape(-1, 2), pattern_points)

In [ ]:
chessboards = [processImage(fn) for fn in img_names]
chessboards = [x for x in chessboards if x is not None]

obj_points = []
img_points = []
for (corners, pattern_points) in chessboards:
        img_points.append(corners)
        obj_points.append(pattern_points)

h, w = cv2.imread(img_names[0], cv2.IMREAD_GRAYSCALE).shape[:2]
rms, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, (w, h), None, None)

print("\nRMS:", rms)
print("camera matrix:\n", camera_matrix)
print("distortion coefficients: ", dist_coefs.ravel())
print("Rotation vectors:", rvecs)
print("translation vectors", tvecs)

processing calibration/chessboards/0.jpg
           calibration/chessboards/0.jpg... OK
processing calibration/chessboards/1.jpg
           calibration/chessboards/1.jpg... OK
processing calibration/chessboards/2.jpg
           calibration/chessboards/2.jpg... OK
processing calibration/chessboards/3.jpg
           calibration/chessboards/3.jpg... OK
processing calibration/chessboards/4.jpg
           calibration/chessboards/4.jpg... OK
processing calibration/chessboards/5.jpg
